In [14]:
import re
import subprocess
import os
import json


In [15]:
exec_name = "cyber"
dir = "out/" + exec_name + "/"
dir_md = dir + "comands-md-files/"


In [16]:
if not os.path.isdir(dir):
    os.mkdir(dir)
    os.mkdir(dir_md)

for f in os.listdir(dir_md):
    os.remove(os.path.join(dir_md, f))


In [17]:
chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
    "Examples:",
]


In [18]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    out = subprocess.run(_args, capture_output=True)
    text = out.stdout.decode("utf-8") + out.stderr.decode("utf-8")

    with open(dir_md + command + ".md", "w") as file:
        file.write(text)

    return text


# text = get_cli_output('cyber')


In [19]:
def parse_cli_stdout(command: str):

    _text = get_cli_output(command)
    _lines = _text.split("\n")

    chapters = {"command": command}
    chapter = "Description"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


# chapters = parse_cli_stdout("gaiad")
# print(chapters)


In [20]:
def get_sub_commands(chapters):
    new_commands = []

    if "Available Commands" in chapters:
        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                sub_command = _match.group(1)

                command_candidate = " ".join(
                    (chapters["command"] + " " + sub_command).split()
                )
                new_commands.append(command_candidate)

    return new_commands
    # del chapters["Available Commands"]


# new_commands = get_sub_commands(chapters)
# new_commands


In [21]:
parsed_commands = []
all_commands_dict_list = []
new_commands = [exec_name + " "]

while True:
    for command in new_commands:

        commands_dict = parse_cli_stdout(command)
        new_command_candidates = get_sub_commands(commands_dict)
        all_commands_dict_list.append(commands_dict)

        for candidate in new_command_candidates:
            if candidate not in parsed_commands:
                new_commands.append(candidate)
                parsed_commands.append(candidate)
        new_commands.remove(command)
    if new_commands == []:
        break


In [22]:
# all_commands_dict_list


In [23]:
json_object = json.dumps(all_commands_dict_list, indent=4)

with open(dir + exec_name + "_commands.json", "w") as outfile:
    outfile.write(json_object)


In [24]:
len(all_commands_dict_list)


284

In [25]:
command_files = []

for f in os.listdir(dir_md):
    if f[-3:] == ".md":
        command_files.append(f)

command_files.sort()
# command_files


In [26]:
if os.path.exists(dir + exec_name + "_commands.md"):
    os.remove(dir + exec_name + "_commands.md")

for f in command_files:
    with open(dir_md + f) as f2:
        text = f2.read()
        file_content = f"### {f[:-3]}\n\n```\n{text}\n```\n\n"

        with open(dir + exec_name + "_commands.md", "a") as file:
            file.write(file_content)
